## Homework3

下記の「#common」で始まるセルの中で指定されたリンク先にある購買データをベースに以下の処理を実行し、問いに答えて課題を提出してください。（データの前処理は、以下の記載の通りに行ってください。）
    
処理内容

1. `#common`のセルで事前に前処理されたデータ`target_online_retail_data_tb`を用います。
2. 購買者（`CustomerID`）の各合計購買金額（`TotalPrice`の合計）を算出し、さらに合計購買金額をベースに降順に並び替えをします。  
3. 合計購買金額の降順に並んだ購買者を$n$等分（$n$はデータ数を越えず、分位数に同一の値が存在しない自然数。購買者数が$n$で割り切れるとは限らず、この場合の処理は`Pandas.qcut`の処理に準じます。）にグループ分けします。
4. それぞれのグループの金額合計値を算出してください（このアプローチをデシル分析といいます）。
5. さらに、それぞれのグループが売上の何割を占めるかを求めてください。全体の売上に対するグループの金額合計値の割合が大きいグループから順に「グループ1」「グループ2」$\cdots$「グループn」とし、`Pandas.Series`形式で回答してください。


(なお、マーケティング戦略では、このように顧客を分けることをセグメンテーションといい、上位2割に絞ってアプローチを仕掛けることをターゲティングといいます。それぞれの戦略によりますが、優良顧客に的を絞った方が投資対効果が高いことが多いため、このようなアプローチを取ることがあります。これに関連して、上位2割の顧客が売上全体の8割を占めることを「パレートの法則」と言います。)

提出するときは、以下の点に注意してください。  
>- 以下の関数`homework`の`!!WRITE ME!!`に処理を書いてください。(**「`!!WRITE ME!!`」は消して、記入してください。**)
>- 実際の提出は記述された`homework`関数全てになり、**提出はOmnicampus内の宿題の欄から今週の課題を選択後、提出内容に関数を貼り付けてから[Pythonコード提出]を押してください。**
>- 前処理は以下のコードを使ってください。  
>- 返り値が`Pandas.Series`形式になるようにしてください。  
>- 解答は、`my_result=pd.Series([0.6,0.2,0.1,0.025,0.025,0.022,0.021,0.02,0.01,0]) `のような形になります。(売上割合はグループ順にして、合計は1になるようにします。)

In [1]:
# common
import pandas as pd
import numpy as np

# init part(データの読み込みと前処理)
file_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
online_retail_data = pd.ExcelFile(file_url)
online_retail_data_table = online_retail_data.parse('Online Retail')

# 採点の都合上、文字列型に変換
online_retail_data_table['cancel_flg'] = online_retail_data_table.InvoiceNo.map(lambda x:str(x)[0])

# InvoiceNoの先頭が5であるものとIDがNullでないものが対象
target_online_retail_data_tb = online_retail_data_table[(online_retail_data_table.cancel_flg == '5') 
                                                        & (online_retail_data_table.CustomerID.notnull())]

target_online_retail_data_tb = target_online_retail_data_tb.assign(TotalPrice=target_online_retail_data_tb.Quantity * target_online_retail_data_tb.UnitPrice)

In [18]:
target_online_retail_data_tb.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,5,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,5,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34


In [19]:
total_price_per_customer_tb = target_online_retail_data_tb.groupby("CustomerID").TotalPrice.sum().sort_values(ascending=False)

In [21]:
total_price_per_customer_qcut = pd.qcut(total_price_per_customer_tb, 3)
total_price_per_customer_qcut

CustomerID
14646.0    (1197.42, 280206.02]
18102.0    (1197.42, 280206.02]
17450.0    (1197.42, 280206.02]
16446.0    (1197.42, 280206.02]
14911.0    (1197.42, 280206.02]
                   ...         
17956.0        (-0.001, 388.58]
16454.0        (-0.001, 388.58]
14792.0        (-0.001, 388.58]
16738.0        (-0.001, 388.58]
13256.0        (-0.001, 388.58]
Name: TotalPrice, Length: 4339, dtype: category
Categories (3, interval[float64, right]): [(-0.001, 388.58] < (388.58, 1197.42] < (1197.42, 280206.02]]

In [24]:
grouped_total_price = total_price_per_customer_tb.groupby(total_price_per_customer_qcut).sum()
grouped_total_price

TotalPrice
(-0.001, 388.58]         318988.910
(388.58, 1197.42]       1030401.213
(1197.42, 280206.02]    7562017.781
Name: TotalPrice, dtype: float64

In [25]:
total_sales = total_price_per_customer_tb.sum()
total_sales 

8911407.904

In [27]:
group_sales_ratio = grouped_total_price / total_sales
group_sales_ratio

In [33]:
sorted_groups = group_sales_ratio.sort_values(ascending=False)
sorted_groups

TotalPrice
(1197.42, 280206.02]    0.848577
(388.58, 1197.42]       0.115627
(-0.001, 388.58]        0.035796
Name: TotalPrice, dtype: float64

In [34]:
sorted_group_labels = ["グループ"+str(i+1)for i in range(3)]

In [37]:
my_result = pd.Series(data=sorted_groups, index=sorted_group_labels)
my_result

グループ1   NaN
グループ2   NaN
グループ3   NaN
Name: TotalPrice, dtype: float64

In [39]:
def homework(target_online_retail_data_tb, n):
    # target_online_retail_data_tbをCustomerIDごとにグループ分けし、TotalPriceの合計値ごとに降順で並び替え
    total_price_per_customer_tb = target_online_retail_data_tb.groupby("CustomerID").TotalPrice.sum().sort_values(ascending = False)
    # pd.qcutで同じ間隔のbinでqcut
    total_price_per_customer_qcut = pd.qcut(total_price_per_customer_tb, n)
    # それぞれのグループの合計値を算出
    grouped_total_price = total_price_per_customer_tb.groupby(total_price_per_customer_qcut).sum()

    # 全体の売上
    total_sales = total_price_per_customer_tb.sum()
    # 全体の売上に対するグループごとの合計金額の割合
    group_sales_ratio = grouped_total_price / total_sales
    # group_sales_ratioの大きい方から降順に並び替え
    sorted_groups = group_sales_ratio.sort_values(ascending=False)
    # pandasのラベルを作成
    sorted_group_labels = ["グループ"+str(i+1)for i in range(n)]
    # ラベルをグループ名、売上割合をdataにとったpd.Seriesをmy_resultとして出力
    my_result = pd.Series(data=sorted_groups, index=sorted_group_labels)

    return my_result

In [42]:
homework(target_online_retail_data_tb, 2)

グループ1   NaN
グループ2   NaN
Name: TotalPrice, dtype: float64

**謝辞**：以下のデータセットの利用に関して  
 https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

引用元：Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [[http://archive.ics.uci.edu/ml](http://archive.ics.uci.edu/ml)]. Irvine, CA: University of California, School of Information and Computer Science.
 
Daqing Chen, Sai Liang Sain, and Kun Guo, Data mining for the online retail industry: A case study of RFM model-based customer segmentation using data mining, Journal of Database Marketing and Customer Strategy Management, Vol. 19, No. 3, pp. 197â€“208, 2012 (Published online before print: 27 August 2012. doi: 10.1057/dbm.2012.17).